In [29]:
#%%appyter init
from appyter import magic
magic.init(lambda _=globals: _())

# Compare Sets Appyter
#### This appyter creates a Venn diagram to visualize the intersections between 2-6 user-inputted gene sets. The user has a choice whether to uplaod a tsv file with the desired genes or to insert the genes into text boxes. 

In [32]:
# Imports
## Venn Diagram
from venn import venn, pseudovenn

## Data Processing
import csv

## Link to Enrichr
import requests
import json
import time
from IPython.display import display, FileLink, Markdown

## Importing for UpSet Plots
from ipywidgets import interact
from upsetjs_jupyter_widget import UpSetWidget

%matplotlib inline

In [30]:
%%appyter hide_code
{% do SectionField(name='section0', title='Compare Gene Sets', subtitle='Create a venn diagram to compare your inputted gene sets', img = 'venndiagram.png') %}
{% do SectionField(name='section1', title='1. Submit Your Gene Lists', subtitle = 'Upload text files containing your gene list -OR- copy and paste your gene list into each text box below (One gene per row).', img = 'bulb.png') %}


## Input Gene Lists

In [31]:
%%appyter code_exec

gs1 = {{ FileField(name = 'gs1', label = 'Gene Set 1 File', default = '', section = 'section1') }}
gs2 = {{ FileField(name = 'gs2', label = 'Gene Set 2 File', default = '', section = 'section1') }}
gs3 = {{ FileField(name = 'gs3', label = 'Gene Set 3 File', default = '', section = 'section1') }}
gs4 = {{ FileField(name = 'gs4', label = 'Gene Set 4 File', default = '', section = 'section1') }}
gs5 = {{ FileField(name = 'gs5', label = 'Gene Set 5 File', default = '', section = 'section1') }}
gs6 = {{ FileField(name = 'gs6', label = 'Gene Set 6 File', default = '', section = 'section1') }}


gs1Text = {{ TextField(name = 'gs1Text', label = 'Gene Set 1', default = '''TRPV1
OPRM1
TRPA1
COMT
SCN9A
TNF
IL6
IL1B
CRP
BDNF
NGF
SLC6A4
MEFV
TRPM8
TRPV4
CALCA
NTRK1
TLR4
ASIC3
SCN10A
MMP9
CNR1
IL10
CCL2
TNNT2
NPPB
PTGS2
CYP2D6
P2RX3
TACR1''', section = 'section1') }}
gs2Text = {{ TextField(name = 'gs2Text', label = 'Gene Set 2', default = '''TNF
TRPV1
CRP
FOS
PTGS2
NGF
TRPA1
BDNF
CD34
POMC
IVD
IL10
ACE
CASP3
CCL2
TLR4
GFAP
TRPM8
IL6
CD68
KIT
OPRM1
SCN9A
CYP2D6
COMT
CEACAM5
GDNF
NPY
PTH
TRPV4''', section = 'section1') }}
gs3Text = {{ TextField(name = 'gs3Text', label = 'Gene Set 3', default = '''OPRD1
TRPV1
TRPA1
SCN9A
OPRM1
TRPM8
TACR1
OPRK1
TAC1
SCN3B
KCNS1
TRPV3
TRPV4
CACNA1B
CACNA2D2
SCN11A
NTRK1
PENK
SCN1B
OPRL1
PDYN
TRPV2
HTR3C
HTR3A
COMT
P2RX3
TRPM5
DRD2
NGFR
FAAH
ASIC3
PNOC
HTR3B
TRPM4
CACNA2D3
BDKRB1
ASIC4
HTR2A
KCNC2
CHRM4
TRPM3
HTR3E
CACNG2
CHRNA7
SCN10A''', section = 'section1') }}
gs4Text = {{ TextField(name = 'gs4Text', label = 'Gene Set 4', default = '', section = 'section1') }}
gs5Text = {{ TextField(name = 'gs5Text', label = 'Gene Set 5', default = '', section = 'section1') }}
gs6Text = {{ TextField(name = 'gs6Text', label = 'Gene Set 6', default = '', section = 'section1') }}

```python
gs1 = ''
gs2 = ''
gs3 = ''
gs4 = ''
gs5 = ''
gs6 = ''
gs1Text = '''BRCA1
BRCA2'''
gs2Text = ''''''
gs3Text = ''''''
gs4Text = ''''''
gs5Text = ''''''
gs6Text = ''''''
```

## Convert Inputs into Lists

In [33]:
%%appyter code_exec
# Helper functions to conver the file upload or text input into gene lists
def file_to_list(str):
    l1 = []
    tsv_file = open(str)
    read_tsv = csv.reader(tsv_file, delimiter = '\t')

    for row in read_tsv:
        l1.append(row[0])

    tsv_file.close()
    return l1

def text_to_list(str):
    l1 = str.splitlines()
    return l1


```python
# Helper functions to conver the file upload or text input into gene lists
def file_to_list(str):
    l1 = []
    tsv_file = open(str)
    read_tsv = csv.reader(tsv_file, delimiter = '\t')
    for row in read_tsv:
        l1.append(row[0])
    tsv_file.close()
    return l1
def text_to_list(str):
    l1 = str.splitlines()
    return l1
```

In [34]:
# Add the appropriate gene lists to the dictionary

gsdict = {}

if gs1 != '':
    l1 = file_to_list(gs1)
    gsdict["Gene Set 1"] = set(l1)

elif gs1Text != '''''':
    l1 = text_to_list(gs1Text)
    gsdict["Gene Set 1"] = set(l1)

if gs2 != '':
    l2 = file_to_list(gs2)
    gsdict["Gene Set 2"] = set(l2)

elif gs2Text != '''''':
    l2 = text_to_list(gs2Text)
    gsdict["Gene Set 2"] = set(l2)

if gs3 != '':
    l3 = file_to_list(gs3)
    gsdict["Gene Set 3"] = set(l3)

elif gs3Text != '''''':
    l3 = text_to_list(gs3Text)
    gsdict["Gene Set 3"] = set(l3)

if gs4 != '':
    l4 = file_to_list(gs4)
    gsdict["Gene Set 4"] = set(l4)

elif gs4Text != '''''':
    l4 = text_to_list(gs4Text)
    gsdict["Gene Set 4"] = set(l4)

if gs5 != '':
    l5 = file_to_list(gs5)
    gsdict["Gene Set 5"] = set(l5)

elif gs5Text != '''''':
    l5 = text_to_list(gs5Text)
    gsdict["Gene Set 5"] = set(l5)

if gs6 != '':
    l6 = file_to_list(gs6)
    gsdict["Gene Set 6"] = set(l6)

elif gs6Text != '''''':
    l6 = text_to_list(gs6Text)
    gsdict["Gene set 6"] = set(l6)


## Draw Venn Diagram

In [35]:
# Generate the venn diagram
venn(gsdict)

if len(gsdict) == 6:
    pseudovenn(gsdict)

ValueError: n_colors must be an integer between 2 and 6